## Exercices Séance 5 : Rappel manipulation de données + Folium

### Echauffement 
A partir de la liste suivante boucler sur les arbres et afficher via format() ou f-string et print() les coordonnées et les types d'arbres

Ex : 'Coordonnées :  48.8736809161, 2.29025241063 et type : sapin'

In [5]:
arbres = [
    {'point':(48.8736809161,2.29025241063), 'type':'sapin'},
    {'point':(48.8577764958,2.31452314446), 'type':'erable'},
    {'point':(48.8678069026,2.38883414897), 'type':'sapin'},
    {'point':(48.8183891866,2.43791851368), 'type':'erable'},
    {'point':(48.8465652331,2.25242001202), 'type':'pin'},
    {'point':(48.857749011,2.33940394029), 'type':'platane'}
]

In [6]:
# Votre code ici : 
for arbre in arbres :
    valeur = f"Coordonnées :  {arbre['point'][0]}, {arbre['point'][1]} et type : {arbre['type']}"
    print(valeur)

Coordonnées :  48.8736809161, 2.29025241063 et type : sapin
Coordonnées :  48.8577764958, 2.31452314446 et type : erable
Coordonnées :  48.8678069026, 2.38883414897 et type : sapin
Coordonnées :  48.8183891866, 2.43791851368 et type : erable
Coordonnées :  48.8465652331, 2.25242001202 et type : pin
Coordonnées :  48.857749011, 2.33940394029 et type : platane


### Créer une carte et ajouter un maker pour chaque point de la liste des arbres

Créer une carte centrée sur Paris avec un fond de plan de votre choix

In [7]:
# ici votre code pour la carte
import folium
map = folium.Map(location=[48.853, 2.35], 
                 zoom_start=12, 
                 tiles='CartoDB positron')
map

### Boucler sur les arbres pour construire les Makers
Ajouter à la carte des markers avec couleur et icon adaptés (icon de feuille + couleur verte).

Cela peut être ajouté à la carte créée juste avant.

Pour cela récupérer les valeurs contenues dans les dictionnaires pour peupler les popups et pour définir les lat lon des markers.

In [8]:
# ici votre code pour ajouter des markers à la carte (dans le cadre du noteboob)
# à partir de la liste des arbres
for arbre in arbres :
    folium.Marker([arbre['point'][0], arbre['point'][1]],
                  popup=arbre['type'],
                  icon=folium.Icon(color='green', icon='leaf')
                 ).add_to(map)    

In [9]:
# ici votre code pour réafficher la carte 
# qui contient maintenant les markers
map

Reprendre le code pour générer une nouvelle carte 'map_detail' dans laquelle les popups contiendront du code html formaté (table) avec le nom de la clés 'type' en gras dans la première ligne et sa valeur dans une seconde ligne.

Utiliser une fonction pour créer le contenu de la popup.

le modèle de code HTML à utiliser est : 

In [20]:
def mapopup(valeur) :
    """
    Fonction qui rempli un template HTML avec une variable texte
    :param valeur: objet de type str
    :return: str code html
    """
    return """
<table style="width: 125px">
    <tr>
        <th>Type d'arbre</th>
    </tr>
    <tr>
        <td>{}</td>
    </tr>
</table>
""".format(valeur)

           
map_detail = folium.Map(location=[48.853, 2.35], 
                        zoom_start=12, 
                        tiles='CartoDB positron')

for arbre in arbres :
    folium.Marker([arbre['point'][0], arbre['point'][1]],
                  popup=mapopup(arbre['type']),
                  icon=folium.Icon(color='green', icon='leaf')
                 ).add_to(map_detail)    
    
map_detail

#### Exercice de synthèse : GeoJson, Dict, List, Chaine, Fonctions, Folium
Charger la donnée des arbres remarquables depuis le fichier Json

Filtrer des objets (ne garder que les Platanes)

Calculer leur emprise

Créer une fonction de popup

Créer une carte vide

Ajouter les markers à la carte avec leurs popups

Initialiser le zoom sur l'emprise calculée

Afficher la carte

Attention à l'ordre des coordonnées dans la création des markers à partir du GeoJson (bien les comparer avec ceux présenté dans les premiers exercices)

Pour cet excercices on va faire des étapes dans des cellules du notebook, on transformera ensuite cela en script python

In [11]:
#Etape Initiale : importer les packages necessaires
import json
import folium

In [12]:
# Etape 1 : Import du fichier GeoJson sous forme de dict python

# chargement du fichier
data = json.load(open('./data/arbresremarquablesparis.geojson', 'r',encoding='utf8'))

In [13]:
# Etape 2 : créer une fonction qui filtre les données selon la valeur d'un attrubut
# (réutiliser la fonction des exercices de la séance 4)

def filtrer(donnee, valeur, attribut):
    resultat = [ feature for feature in donnee["features"] if feature["properties"][attribut] == valeur]
    return resultat


In [14]:
# Etape 3 : Créer une nouvelle fonction qui va calculer l'emprise des données

def emprise(features):
    # dans la boucle que l'on va traiter le premier objet de manière spécifique
    # pour initialiser l'emprise avec ses coordonnées
    # on utilise un booleen pour marquer le passage sur le premier éléments
    premier = True
    
    for feature in features:
        coord = feature["geometry"]["coordinates"]
        if premier == True:
            # si on est sur le premier point on va utiliser les coordonnées pour initialiser
            # les 4 valeurs formant l'emprise
            xmin = coord[0]
            xmax = coord[0]
            ymin = coord[1]
            ymax = coord[1] 
            # et on passe le marquer à False pour ne plus passer par cette partie du code pour les autres features
            premier = False
        else :
            # on traite maintenant une feature qui n'est pas la première
            # on doit étudier si elle doit modifier l'emprise

            # on compare le x de la feature au xmin
            if coord[0] < xmin:
                xmin = coord[0]
            # puis au Xmax
            if coord[0] > xmax:
                xmax = coord[0]  
            # puis le Y au ymin
            if coord[1] < ymin:
                ymin = coord[1]
            # puis au Ymax
            if coord[1] > ymax:
                ymax = coord[1]
    # ayant lu la doc Folium
    # m.fit_bounds([[52.193636, -2.221575], [52.636878, -1.139759]])
    return [[ymin, xmin], [ymax, xmax]]
        

In [15]:
# Etape 4 : faire une fonction pour créer une popop à partir 
# de toutes les properties d'un objet
def mapopup_arbre(properties):
    html_table = "<table style='width: 200px'>"
    html_fintable = "</table>"
    html = ""
    for key, value in properties.items():
        html += "<tr><th>{}</th><td>{}</td></tr>".format(key, value)

    return html_table + html + html_fintable

In [16]:
## Affichage des données issues des deux fonctions :
resultat = filtrer(data, 'Sophora', 'libellefrancais')
geoemprise = emprise(resultat)

In [17]:
# Etape 5 : Créer la carte, y ajouter les markers pour les objets filtrés, appliquer le zoom sur l'emprise calculée

# création de l'objet carte avec un centre et un zoom par défaut mais qui sera modifié plus bas pas l'emprise
map_finale = folium.Map(location=[48.853, 2.35], 
                        zoom_start=12, 
                        tiles='CartoDB positron')

for arbre in resultat :
    # on récupère les coordonnées de l'arbre
    coord = arbre["geometry"]["coordinates"]
    # Attention à l'odre des coordonnées, bien analyser le GeoJson
    folium.Marker([coord[1], coord[0]],
                  popup=mapopup_arbre(arbre['properties']),
                  icon=folium.Icon(color='green', icon='leaf')
                 ).add_to(map_finale) 

map_finale.fit_bounds(bounds=geoemprise)

In [18]:
# etape intermédiaire : voir la carte
map_finale

In [19]:
# Etape 7 enregistrer la carte

map_finale.save("Seance_05_index.html")

### Si vous voulez ...

Choisir une autre source de données GeoJson (sur un site OpenData) et refaire le déroulé de l'exercice de synthèse.

Essayer en prenant un Geojson qui n'est pas composé de géométries de types points, ajuster les fonctions développées à l'exercice de synthèse pour répondre aux différents types de géométrie.

Consulter la documentation Folium pour la création de géométries autres que ponctuelles.


**Exemple proposé :** 

https://opendata.paris.fr/explore/dataset/stationnement-sur-voie-publique-emprises

Filtrer sur les emplacements de 2 roues de plus de 10 places et zoomer dessus